In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/ECE421_Lab3/')
import helper as hlp


# Loading data
def load_dataset(is_2D,is_valid = False):
  data = None
  if(is_2D):
    data = np.load('/content/drive/MyDrive/Colab Notebooks/ECE421_Lab3/data2D.npy')
  else:
    data = np.load('/content/drive/MyDrive/Colab Notebooks/ECE421_Lab3/data100D.npy')
  [num_pts, dim] = np.shape(data)
  print("num_pts = ",num_pts,"  dim = ",dim)

  # For Validation set
  if is_valid:
    valid_batch = int(num_pts / 3.0)
    np.random.seed(45689)
    rnd_idx = np.arange(num_pts)
    np.random.shuffle(rnd_idx)
    val_data = data[rnd_idx[:valid_batch]]
    data = data[rnd_idx[valid_batch:]]
    return data, val_data
  else:
    return data


# Distance function for K-means
def distance_func(X, mu):
  """ Inputs:
        X: is an NxD matrix (N observations and D dimensions)
        mu: is an KxD matrix (K means and D dimensions)
        
      Output:
        pair_dist: is the squared pairwise distance matrix (NxK)
  """
  X = tf.expand_dims(X,1) #shape becomes (N,1,D)
  square = tf.square((X-mu))
  pair_dist = tf.reduce_sum(square,2) #change shape to (N,K)
  
  return pair_dist

#============================== Helper ===========================================
def reduce_logsumexp(input_tensor, reduction_indices=1, keep_dims=False):
  """Computes the sum of elements across dimensions of a tensor in log domain.

     It uses a similar API to tf.reduce_sum.

  Args:
    input_tensor: The tensor to reduce. Should have numeric type.
    reduction_indices: The dimensions to reduce. 
    keep_dims: If true, retains reduced dimensions with length 1.
  Returns:
    The reduced tensor.
  """
  max_input_tensor1 = tf.reduce_max(
      input_tensor, reduction_indices, keep_dims=keep_dims)
  max_input_tensor2 = max_input_tensor1
  if not keep_dims:
    max_input_tensor2 = tf.expand_dims(max_input_tensor2, reduction_indices)
  return tf.log(
      tf.reduce_sum(
          tf.exp(input_tensor - max_input_tensor2),
          reduction_indices,
          keep_dims=keep_dims)) + max_input_tensor1


def logsoftmax(input_tensor):
  """Computes normal softmax nonlinearity in log domain.

     It can be used to normalize log probability.
     The softmax is always computed along the second dimension of the input Tensor.     

  Args:
    input_tensor: Unnormalized log probability.
  Returns:
    normalized log probability.
  """
  return input_tensor - reduce_logsumexp(input_tensor, reduction_indices=0, keep_dims=True)

#============================================= GMM ================================================

def log_gauss_pdf(X, mu, sigma):
  """ Inputs: 
      X: N X D
      mu: K X D
      sigma: K X 1

    Outputs:
      log Gaussian PDF (N X K)
  """
  part1 = -(0.5) * tf.log(2 * np.pi * tf.transpose(sigma)**2)
  pair_dist = distance_func(X, mu)
  part2 = -(0.5) * tf.square(pair_dist) / (tf.transpose(sigma)**2)
  return part1 + part2

def log_posterior(log_PDF, log_pi):
  """ Inputs:
      log_PDF: log Gaussian PDF N X K
      log_pi: K X 1

    Outputs
      log_post: N X K
  """
  log_numerator = log_PDF + tf.squeeze(log_pi)
  log_denominator = reduce_logsumexp(log_numerator, keep_dims=True)
  answer = log_numerator - log_denominator
  return answer



def build_GMM_Graph(K,D):
  #define variables
  X = tf.placeholder(tf.float32, shape=(None,D))
  mu = tf.Variable(tf.random_normal([K, D], stddev = 1)) 
  # theta is unconstrained parameter, sigma = exp(phi) with [0 - inf] 
  theta = tf.Variable(tf.random_normal([K, 1], stddev = 1))
  sigma = tf.exp(theta)

  # phi is unconstrained parameter, acheive constraint for pi
  phi = tf.Variable(tf.random_normal([K, 1], stddev = 1))
  log_pi = logsoftmax(phi)

  log_pdf = log_gauss_pdf(X,mu,sigma)

  #defien loss & optimizer
  loss= - tf.reduce_sum(reduce_logsumexp(log_pdf + tf.squeeze(log_pi), keep_dims=True))
  optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5).minimize(loss)
  return X,mu,sigma,optimizer,loss,log_pdf,log_pi

def Train_GMM(dataset,K,epochs,valid_data):
  D = dataset.shape[1]
  X,mu,sigma,optimizer,loss,log_pdf,log_pi = build_GMM_Graph(K,D)
  loss_List = []
  best_mu = None
  best_sigma = None
  best_pi = None
  cluster = None
  valid_loss = None
  with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      for i in range(epochs):
          best_mu,best_sigma,best_pi,opt = sess.run([mu,sigma,log_pi,optimizer], feed_dict={X: dataset})
          Loss = sess.run(loss, feed_dict={X: dataset})
          loss_List.append(Loss/dataset.shape[0])

      # end of traning find cluster
      # Returns the index with the smallest value across axes of a tensor
      cluster = sess.run(tf.argmax(log_posterior(log_pdf,log_pi),1),feed_dict={X:dataset})
      # end of training find validation loss
      valid_loss = sess.run(loss, feed_dict={X: valid_data})
      valid_loss = valid_loss/valid_data.shape[0]
  return loss_List,best_mu,best_sigma,best_pi,cluster,valid_loss

#============================================= GMM ================================================

def build_k_means_Graph(K,D):
  X = tf.placeholder(tf.float64, shape=(None,D))
  mu = tf.Variable(tf.truncated_normal((K, D), mean=0, stddev=1, dtype=tf.float64), trainable=True)
  loss = tf.reduce_sum(tf.reduce_min(distance_func(X,mu), axis=1))
  optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.9, beta2=0.99, epsilon=1e-5).minimize(loss)
  return X,mu,loss,optimizer


def Train_K_means(dataset,K,epochs,valid_data):
  D = dataset.shape[1]
  X,mu,loss,optimizer = build_k_means_Graph(K,D)
  loss_List = []
  best_mu = None
  cluster = None
  valid_loss = None
  with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      for i in range(epochs):
          best_mu,opt = sess.run([mu,optimizer], feed_dict={X: dataset})
          Loss = sess.run(loss, feed_dict={X: dataset})
          loss_List.append(Loss/dataset.shape[0])

      # end of traning find cluster
      # Returns the index with the smallest value across axes of a tensor
      cluster = sess.run(tf.argmin(distance_func(X, best_mu), 1),feed_dict={X:dataset})
      # end of training find validation loss
      valid_loss = sess.run(loss, feed_dict={X: valid_data})
      valid_loss = valid_loss/valid_data.shape[0]
  return loss_List,best_mu,cluster,valid_loss


#============================================ Test ===============================================
def Q3():
  K_list = [5,10,15,20,30]
  k_mean_valid_loss = None
  GMM_valid_loss = None
  for K in K_list:
    data, val_data = load_dataset(False,True)
    loss_List,best_mu,cluster,           k_mean_valid_loss = Train_K_means(data,K,100,val_data)
    loss_List,best_mu,best_sigma,best_pi,cluster,  GMM_valid_loss = Train_GMM(data,K,100,val_data)
    print("K=",K,"   |k_mean_valid_loss =",k_mean_valid_loss,"   |GMM_valid_loss=",GMM_valid_loss)
  return  

Q3()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
num_pts =  10000   dim =  100
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
K= 5    |k_mean_valid_loss = 21.54334444037717    |GMM_valid_loss= 10.834202951545155
num_pts =  10000   dim =  100
K= 10    |k_mean_valid_loss = 21.24050911505024    |GMM_valid_loss= 12.697508813381338
num_pts =  10000   dim =  100
K= 15    |k_mean_valid_loss = 21.042035154237134    |GMM_valid_loss= 9.654315040879087
num_pts =  10000   dim =  100
K= 20    |k_mean_valid_loss = 20.992872165406673    |GMM_valid_loss= 8.715772553817882
num_pts =  10000   dim =  100
K= 30    |k_mean_valid_loss = 20.71745446427862    |GMM_valid_loss= 6.316014218609361


# 新段落

# 新段落

# 新段落